In [1]:
import numpy as np

### Introduction
Differential equation describing the system.

$$J\frac{d\omega}{dt} = M_e - M_0 - M_{LOAD}$$
where:
- $M_e$: electromagnetic moment
- $M_0$: braking moment
- $M_{LOAD}$: load moment
- $J$: moment of inertia

Transformation to differential form using the Eulerian method.
$$J\frac{d\omega(t+\Delta t) - \omega (t)}{\Delta t} = M_e(t)-M_0-M_{LOAD}(t)$$
$$\omega (t+\Delta t) = \omega(t) + \frac{\Delta t}{J}(M_e(t)- M_0-M_{LOAD}(t))$$

### Defined Parameters

In [ ]:
timeOfSimulation = 30
timeOfSample = 0.1

In [ ]:
def calculateNumberOfSteps(timeOfSimulation: int, timeOfSample: float) -> int:
    """ This function calculates number of iterations for simulation of process

        Parameters:
        - total_time (int): total time of simulation in seconds
        - sammpling_time (float): time at which we repeat the measurement in seconds

        Returns:
        - int: number of iterations
    """
    return int(timeOfSimulation / timeOfSample) + 1